# UGIT Test Routines

In [52]:
import textwrap

from ugit import data
from ugit import base

In [26]:
def refs_test (oid):
    """ Expects an Commit Object Id """
    print ('''
Testing Reference Functions:
- ie. get_ref(), update_ref(), delete_ref() & iter_refs, 
''')

    ref = f'refs/tags/test1'

    print ('''Set a new reference: test1
- and retrieve its value') 
''')
    data.update_ref (ref, data.RefValue (symbolic=False, value=oid))
    print (data.get_ref (ref).value)

    print ('''
Fetch all references and print their OIDs
''')
    for i in data.iter_refs (): print (i[1][1])

    print ('''
  - and delete the reference again''')
    data.delete_ref (ref)

In [27]:
def show_index ():
    print ('''
Retrieving the content from the json file: .ugit/index
''')
    with data.get_index() as index: print( index)

In [28]:
def push_test (oid, remote_dir):
    """ Expects Object Id & destination path """
    print (' Testing change_git_dir() together with push_object() & get_object()')

    # oid = '47729ee8498ece441d20d936c2efb4a9c56a3cc7'
    # remote_dir = '/home/klerai/tmp1'

    print (' - insuring that the object database is available in the remote directory')
    with data.change_git_dir (remote_dir):
        data.init()

    print (' - pushing existing object to remote store,')
    data.push_object (oid, remote_dir)

    print (' - afterwards displaying 25 chars of its content from the remote store')
    print ()
    with data.change_git_dir (remote_dir):
        print( data.get_object (oid).decode()[:25])

## Base Layer Test

In [29]:
def populate_db ():
    """ Initial setup for the UGIT database """
    if data.GIT_DIR.exists():
        print ('Please remove existing .ugit tree before you run this test!')
    else:
        print ('Setting up the original .ugit dabase structure')
        base.init ()
        print (' - adding the ugit directory to the database ')
        base.add (['ugit'])
        print (' - and commit it')
        oid = base.commit ('Initial commit for the files in ugit/')
        print (' - attaching a branch to it')
        base.create_branch ('v0.1', oid)
        print (' - adding some more files')
        base.add (['setup.py','WorkNotes.txt'])
        print (' - with commit')
        oid = base.commit ('added helper files to the working directort')
        print (' - and attaching a tag to it')
        base.create_tag ('Stage1', oid)
        base.commit ('Working directory is prepared')
        print ('we are ready to go')

In [54]:
def check_HEAD ():
    HEAD = base.get_oid ('@')
    c_HEAD = base.get_commit (HEAD)
    print (f'''
TEST: Fetching current HEAD's OID:
      {HEAD}

      - and retrieving it's commit record:
      
{c_HEAD}

      - followed by the tree object with list of files
''')
    print (base.get_tree (c_HEAD[0]))

In [55]:
populate_db ()
check_HEAD ()

Please remove existing .ugit tree before you run this test!

TEST: Fetching current HEAD's OID:
      f0ed05c972c1336315641d3cabd6169bb8b9b2c1

      - and retrieving it's commit record:
      
Commit(tree='992bcf6e9e337d4fb01e9adedbead7755ab41ddf', parents=['a5b1ef8fabf605b1a50b88c17a87f7440f662bd6'], message='Working directory is prepared')

      - followed by the tree object with list of files

{'WorkNotes.txt': '52640e3e3a7f26dbbf38f78d5ae85936b9ed4866', 'setup.py': '2755dd23451d7d50751db5cacb3b059dc076989c', 'ugit/4tests/list_object_heads.py': 'bbb490717ee7274f8c1c4c7104d23a7e159419c4', 'ugit/4tests/ugit-test.py': '2b62fa7d22412ba4348f71b979b97bb747785b0c', 'ugit/__pycache__/base.cpython-36.pyc': '69603150f4ecdb5a811cc8fbbe53b93c1a65f2d4', 'ugit/__pycache__/data.cpython-36.pyc': '8791ff03518dca0f8521ba7d6dce987aeee58346', 'ugit/__pycache__/diff.cpython-36.pyc': 'b9ddb27dae2e6ed6396896e98d18388bb4c84512', 'ugit/base.py': '82d611a6bbf08c9a81f11ab00ab7da781dac4a9e', 'ugit/cli.py': '

In [32]:
show_index()


Retrieving the content from the json file: .ugit/index

{'WorkNotes.txt': '52640e3e3a7f26dbbf38f78d5ae85936b9ed4866', 'setup.py': '2755dd23451d7d50751db5cacb3b059dc076989c', 'ugit/4tests/list_object_heads.py': 'bbb490717ee7274f8c1c4c7104d23a7e159419c4', 'ugit/4tests/ugit-test.py': '2b62fa7d22412ba4348f71b979b97bb747785b0c', 'ugit/__pycache__/base.cpython-36.pyc': '69603150f4ecdb5a811cc8fbbe53b93c1a65f2d4', 'ugit/__pycache__/data.cpython-36.pyc': '8791ff03518dca0f8521ba7d6dce987aeee58346', 'ugit/__pycache__/diff.cpython-36.pyc': 'b9ddb27dae2e6ed6396896e98d18388bb4c84512', 'ugit/base.py': '82d611a6bbf08c9a81f11ab00ab7da781dac4a9e', 'ugit/cli.py': 'b2838bba4356dcaad890f01028d95492ce22ee54', 'ugit/data.py': 'a434464f5e65b68695bd27fc1d29a8d18f927b8a', 'ugit/diff.py': '67ee81b8e3c4c94c2bc70d741f21dfb997a493c9', 'ugit/remote.py': '47aed01fb9e6d9774f65074d207481af4aa73161'}


In [33]:
print (' - list all references')
for ref in data.iter_refs (): print (ref[0])

 - list all references
HEAD
refs/heads/v0.1
refs/heads/master
refs/tags/Stage1


In [34]:
print (' - and listing all branches')
for name in base.iter_branch_names (): print (name)

 - and listing all branches
v0.1
master


In [53]:
def _print_commit (oid, commit, refs=None):
    refs_str = f' ({", ".join (refs)})' if refs else ''
    print (f'commit {oid}{refs_str}\n')
    print (textwrap.indent (commit.message, '    '))
    print ('')

def log (arg_oid):
    refs = {}
    for refname, ref in data.iter_refs ():
        refs.setdefault (ref.value, []).append (refname)

    for oid in base.iter_commits_and_parents ({arg_oid}):
        commit = base.get_commit (oid)
        _print_commit (oid, commit, refs.get (oid))

log (base.get_oid('@'))

commit f0ed05c972c1336315641d3cabd6169bb8b9b2c1 (HEAD, refs/heads/master)

    Working directory is prepared

commit a5b1ef8fabf605b1a50b88c17a87f7440f662bd6 (refs/tags/Stage1)

    added helper files to the working directort

commit 1374e905cf08bf82807f9466ee7c08290b648952 (refs/heads/v0.1)

    Initial commit for the files in ugit/



In [36]:
#base.checkout ('v0.1')

In [37]:
#base.checkout ('master')

In [38]:
refs_test ('d837bc638b07078f86bb7e00b901971723f1a671')


Testing Reference Functions:
- ie. get_ref(), update_ref(), delete_ref() & iter_refs, 

Set a new reference: test1
- and retrieve its value') 

d837bc638b07078f86bb7e00b901971723f1a671

Fetch all references and print their OIDs

f0ed05c972c1336315641d3cabd6169bb8b9b2c1
1374e905cf08bf82807f9466ee7c08290b648952
f0ed05c972c1336315641d3cabd6169bb8b9b2c1
a5b1ef8fabf605b1a50b88c17a87f7440f662bd6
d837bc638b07078f86bb7e00b901971723f1a671

  - and delete the reference again


In [51]:
print( base.get_oid ('v0.1'))

1374e905cf08bf82807f9466ee7c08290b648952


In [40]:
if (data.GIT_DIR / 'refs/heads/master').is_file(): print ('Ok')

Ok


In [ ]:
# cli.cat_file('f0ed05c972c1336315641d3cabd6169bb8b9b2c1')